# Example of Staging Upload to ImageCalc Table
This notebook goes through an example of uploading a classified planet image, a logfile, and training data to the DSWx calval database. Data files and metadata are uploading to a staging bucket. Then, the database manager will commit these uploads to the database

In [ ]:
import geopandas as gpd
import pandas as pd
import boto3
import os
from datetime import datetime
import sys
sys.path.insert(0, './tools/')
from addImageCalc import addImageCalc

### AWS Credentials
In order to download imagery from the private bucket, JPL RSA access and OPERA Calval AWS credenitals are needed

In [ ]:
bucket_name = 'opera-calval-database-dswx'

In [ ]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

### Read Image metadata table
To get the geometry metadata for the classified image, we copy the geometry of the source image from the database since the extents are the same. This geometry could also be generated directly from the classified imagery.

In [ ]:
imageTable = gpd.read_file(s3.Object(bucket_name,'image.geojson').get()['Body'])
imagecalcTable = gpd.read_file(s3.Object(bucket_name,'image_calc.geojson').get()['Body'])

In [ ]:
geometry = imageTable[imageTable.image_name == '20210912_034049_22_2421'].geometry.iloc[0]
now = datetime.now().strftime("%Y%m%d_%H%M%S")


### Enter the required file locations and metadata fields
To upload the classified image, we need to specify its location on the local computer (and the location of auxilary files). We also need to fill in some metadata fields. Both file paths and metadata are specified as dictionaries

In [ ]:
filePaths = {
    'image_calc' : '/Users/mbonnema/Documents/OPERA_calval/DSWx/Chip_4_8_QGIS/files/20210912_034049_22_2421_3B_AnalyticMS_SR_8b_clip_supMD_Alg.tif',
    'logfile' : '/Users/mbonnema/Documents/OPERA_calval/DSWx/Chip_4_8_QGIS/files/0210912_034049_22_2421_logfile.docx',
    'training' : '/Users/mbonnema/Documents/OPERA_calval/DSWx/Chip_4_8_QGIS/files/training.scp'
}


In [ ]:
metaData = {
    'image_name':'20210912_034049_22_2421', #str
    'image_calc_name':'20210912_034049_22_2421_supMD', #str
    'previous_name':None, #str
    'calc_type':'Supervised Classification', #str
    'processing_level':'Intermediate', #str
    'oversight_level':None, #str
    'fitness':None, #str
    'public':True, #bool
    'geometry':geometry, #shapely geometry
}
metaData

### Stage the image
We use a pre-defined function to upload files and metadata to the staging area. This function takes the file paths and metadata dictionaries, as well as the AWS session object as inputs

    

In [ ]:
addImageCalc(filePaths,metaData,session)